In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets,transforms

In [ ]:
from torch.utils.data import DataLoader 

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
import numpy as np


In [23]:
# Transforms

In [24]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [25]:
# Dataset + Filter

In [26]:
import certifi
print(certifi.where()) 

/Users/ortalhanuna/my-code/.venv/lib/python3.13/site-packages/certifi/cacert.pem


In [27]:
train_dataset = datasets.CIFAR10(root='./data', train=True,download=True,transform=transform)
test_dataset =  datasets.CIFAR10(root ='./data', train=False,download=True,transform=transform)

In [28]:
print(train_dataset.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [29]:
def filter_cars_only(ds):
    mask = np.isin(ds.targets,[1,9])
    ds.data , ds.targets = ds.data[mask],np.array(ds.targets)[mask]
    ds.targets = [0 if y==1 else 1 for y in ds.targets if y in [1,9]]
    return ds

print(train_dataset)
train_dataset = filter_cars_only(train_dataset)
print(train_dataset)

test_dataset = filter_cars_only(test_dataset)


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [30]:
train_loader = DataLoader(train_dataset, batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False)

In [31]:
classes = {0:'automobile', 1: 'truck'}

In [32]:
# model

In [33]:
class QuantizedVehicleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.quant = torch.quantization.QuantStub()

        self.conv1 = nn.Conv2d(3,16,3,1)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(16,32,3,1)
        self.relu2 = nn.ReLU()
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(32*14*14,64)
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(64,3)         

        self.dequant = torch.quantization.DeQuantStub()

    def forward(self,x):
        x = self.quant(x)            
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.dequant(x)         
        return x

# torch.Size([64, 3, 32, 32])
# torch.Size([64, 3, 32, 32])
# torch.Size([64, 16, 30, 30])
# torch.Size([64, 16, 30, 30])
# torch.Size([64, 32, 28, 28])
# torch.Size([64, 32, 28, 28])
# torch.Size([64, 32, 14, 14])
# torch.Size([64, 6272])
# torch.Size([64, 64])
# torch.Size([64, 64])
# torch.Size([64, 3])
# torch.Size([64, 3])

In [34]:
model = QuantizedVehicleCNN()


In [35]:
## loss + optimizer

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [37]:
# train

In [39]:
epochs = 50
for epoch in range(epochs):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs,y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
 # x-pics ,  y-real_results,   outputs-model_results

Epoch 1, Loss: 0.4537
Epoch 2, Loss: 0.2510
Epoch 3, Loss: 0.3068
Epoch 4, Loss: 0.1156
Epoch 5, Loss: 0.1509
Epoch 6, Loss: 0.0385
Epoch 7, Loss: 0.0611
Epoch 8, Loss: 0.0309
Epoch 9, Loss: 0.1309
Epoch 10, Loss: 0.0124
Epoch 11, Loss: 0.0028
Epoch 12, Loss: 0.0135
Epoch 13, Loss: 0.0021
Epoch 14, Loss: 0.0028
Epoch 15, Loss: 0.0043
Epoch 16, Loss: 0.0004
Epoch 17, Loss: 0.0001
Epoch 18, Loss: 0.0014
Epoch 19, Loss: 0.0003
Epoch 20, Loss: 0.0004
Epoch 21, Loss: 0.0006
Epoch 22, Loss: 0.0001
Epoch 23, Loss: 0.0003
Epoch 24, Loss: 0.0002
Epoch 25, Loss: 0.0003
Epoch 26, Loss: 0.0002
Epoch 27, Loss: 0.0001
Epoch 28, Loss: 0.0001
Epoch 29, Loss: 0.0002
Epoch 30, Loss: 0.0002
Epoch 31, Loss: 0.0003
Epoch 32, Loss: 0.0000
Epoch 33, Loss: 0.0001
Epoch 34, Loss: 0.0000
Epoch 35, Loss: 0.0002
Epoch 36, Loss: 0.0001
Epoch 37, Loss: 0.0000
Epoch 38, Loss: 0.0001
Epoch 39, Loss: 0.0001
Epoch 40, Loss: 0.0000
Epoch 41, Loss: 0.0000
Epoch 42, Loss: 0.0001
Epoch 43, Loss: 0.0000
Epoch 44, Loss: 0.00

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)
        _, preds = torch.max(outputs,1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)
print("Test Accuracy :", correct/total)

In [41]:
import torch

x = torch.arange(6)       # [0,1,2,3,4,5]
y = x.view(2,3)           # טוב, כל הנתונים רציפים
print(y)

z = x.t().reshape(2,3)    # אם הטנזור לא רציף, reshape() ייצור עותק
print(z)


tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 1, 2],
        [3, 4, 5]])
